In [ ]:
########################################
# meta knn-* using all of raw features #
########################################
import numpy as np
import pandas as pd
import datetime
import time
import os,sys
import gc
from sklearn import *
import math

def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred) ** 0.5

drop_cols = ['id', 'visit_date', 'visitors', 'hpg_store_id', 'fold', 'air_store_id', 
             'air_store_id_encoded', 'hpg_store_id_encoded', 
             'air_reserved_visitors', 'hpg_reserved_visitors','reserved_visitors']

DataBaseDir = '../../data'
InputDir = '%s/l0/kfold' % DataBaseDir
OutputDir = '%s/meta/kfold' % DataBaseDir
kfold = 5
neighor_range = 6
use_selected = False
neighbor_num_list = [int(math.pow(2, i)) for i in list(range(1, neighor_range + 1))]
#### load data
valid_dfs = []
holdout_dfs = []
test_dfs = []
for fold in range(kfold):
    FoldInputDir = '%s/%s' % (InputDir, fold)
    valid = pd.read_csv('%s/valid.csv' % FoldInputDir, parse_dates= ['visit_date']).reset_index(drop= True)
    holdout = pd.read_csv('%s/holdout.csv' % FoldInputDir, parse_dates= ['visit_date']).reset_index(drop= True)
    test = pd.read_csv('%s/test.csv' % FoldInputDir, parse_dates= ['visit_date']).reset_index(drop= True)
    valid['fold'] = fold
    valid_dfs.append(valid)
    holdout_dfs.append(holdout)
    test_dfs.append(test)
TrainData = pd.concat(valid_dfs, axis= 0, ignore_index= True) 
##### model selection with CV
start = time.time()
for nn in neighbor_num_list:
    strategy = 'knn_%s' % nn
    # score
    cv_score = .0
    holdout_score = .0
    for fold in range(kfold):
        FoldData = {
            'train': TrainData[TrainData['fold'] != fold],
            'valid': TrainData[TrainData['fold'] == fold],
            'holdout': holdout_dfs[fold],
            'test': test_dfs[fold]
        }
        if(use_selected):
            selected_features = []
            with open('../../data/gfs/en_good_features.txt', 'r') as i_file:
                for line in i_file:
                    selected_features.append(line.rstrip())
            i_file.close()
#             selected_features.extend(meta_feats)
            #selected_features.extend(['air_store_id_encoded', 'hpg_store_id_encoded'])
            col = selected_features
        else:
            col = [c for c in FoldData['train'].columns if c not in drop_cols]
            col = [c for c in col if((c.startswith('inter') == False))]
        print('feature size %s' % len(col))
        model = neighbors.KNeighborsRegressor(n_jobs= 2, n_neighbors= nn)
        model.fit(FoldData['train'][col].values, FoldData['train']['visitors'].values)
        # for valid
        FoldData['valid'][strategy] = model.predict(FoldData['valid'][col].values)
        rmsle_valid = RMSLE(FoldData['valid']['visitors'].values, FoldData['valid'][strategy])
        cv_score += rmsle_valid
        # for holdout
        FoldData['holdout'][strategy] = model.predict(FoldData['holdout'][col])
        rmsle_holdout = RMSLE(FoldData['holdout']['visitors'].values, FoldData['holdout'][strategy])
        holdout_score += rmsle_holdout
        # for test
        FoldData['test'][strategy] = model.predict(FoldData['test'][col])
        #### output
        FoldOutputDir = '%s/%s' % (OutputDir, fold)
        if(os.path.exists(FoldOutputDir) == False):
            os.makedirs(FoldOutputDir)
        for mod in FoldData.keys():
            if(mod == 'train'):
                continue
            OutCols = []
            if(mod == 'test'):
                OutCols.append('id')
            OutCols.extend(['air_store_id', 'visit_date', 'visitors', strategy])
            OutputFile = '%s/%s_%s.csv' % (FoldOutputDir, mod, strategy)
            OutFoldData = FoldData[mod][OutCols]
            OutFoldData.to_csv(OutputFile, index= False)
        print('fold %s: valid score %.6f, holdout score %.6f, valid length %s' % (fold, rmsle_valid, rmsle_holdout, len(FoldData['valid'])))  
    cv_score /= kfold # Average valid set predictions
    holdout_score /= kfold # Average holdout set predictions
    end = time.time()
    print('\n======================')
    print("meta model KNN-%s done, CV score %.4f, Holdout score %.4f, Elapsed time: %.2fs" % (nn, cv_score, holdout_score, (end - start)))
    print('======================\n')

/Users/yuanpingzhou/miniconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/yuanpingzhou/miniconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/Users/yuanpingzhou/miniconda3/lib/python3.6/site-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module

feature size 222


/Users/yuanpingzhou/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


fold 0: valid score 0.751364, holdout score 0.751868, valid length 45371
feature size 222
fold 1: valid score 0.754002, holdout score 0.751541, valid length 45371
feature size 222
fold 2: valid score 0.753310, holdout score 0.753107, valid length 45371
feature size 222
fold 3: valid score 0.754123, holdout score 0.745374, valid length 45370
feature size 222
fold 4: valid score 0.757228, holdout score 0.753443, valid length 45370

meta model KNN-2 done, CV score 0.7540, Holdout score 0.7511, Elapsed time: 417.88s

feature size 222
fold 0: valid score 0.703998, holdout score 0.700889, valid length 45371
feature size 222
